In [1]:
#!pip install snowflake-connector-python pandas
#!pip install snowflake-connector-python
#!pip install snowflake-sqlalchemy
#!pip install sqlalchemy
#!pip install snowflake-connector-python snowflake-sqlalchemy
#!pip install geopy
#!pip install ortools
#!pip install networkx matplotlib folium
#!pip install polyline


In [2]:
import pandas as pd
import numpy as np
import snowflake.connector
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import getpass
from snowflake.connector.pandas_tools import pd_writer
from snowflake.connector.pandas_tools import write_pandas
import googlemaps
import folium
from folium.plugins import MarkerCluster
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import asyncio
import aiohttp
import nest_asyncio
from scipy.spatial.distance import cdist
from geopy.distance import great_circle
import matplotlib.pyplot as plt
import time
import requests
from geopy.geocoders import Nominatim
from geopy.distance import great_circle
import networkx as nx
from folium.plugins import AntPath
import itertools
from geopy.distance import geodesic
from sklearn.preprocessing import StandardScaler
import warnings


In [3]:
nest_asyncio.apply()
file_path = '/Users/vidhuprabha/Downloads/UPS_Facilities.csv'

ups_facilities = pd.read_csv(file_path)

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

fac_tab = 'FAC_DATA_TB'
loc_tab = 'FAC_LOC_TB'
cord_tab = 'FAC_CORD_TB'

conn = snowflake.connector.connect(
    user='VIDHUPRABHA',
    password='Vids@1231',
    account='iwcpjnq-wb90177',
    warehouse='FACILITY_WH',
    database='FACILITY_DB',
    schema='FACILITY_SC'
    )

In [5]:
def pre_process_locations(ups_facilities):
    allowed_facilities = [
        'UPS Alliance Location',
        'The UPS Store',
        'UPS Customer Center'
    ]

    fac_data = ups_facilities.drop_duplicates()

    fac_data.fillna(method='ffill', inplace=True)

    if 'ADDRESS' in fac_data.columns:
        fac_data['ADDRESS'] = fac_data['ADDRESS'].str.upper()

    if 'LATITUDE' in fac_data.columns and 'LONGITUDE' in fac_data.columns:
        fac_data = fac_data[
            (fac_data['LATITUDE'] >= -90) & (fac_data['LATITUDE'] <= 90) &
            (fac_data['LONGITUDE'] >= -180) & (fac_data['LONGITUDE'] <= 180)
        ]

    if 'ZIP' in fac_data.columns:
        fac_data['ZIP'] = fac_data['ZIP'].astype(str).str.zfill(5)

    required_columns = ['NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'LATITUDE', 'LONGITUDE']
    if all(col in fac_data.columns for col in required_columns):
        loc_data = fac_data[required_columns].drop_duplicates(subset=['LATITUDE', 'LONGITUDE'])
    else:
        loc_data = pd.DataFrame(columns=required_columns)

    loc_data = loc_data[loc_data['NAME'].isin(allowed_facilities)]
    
    cord_data = loc_data[['LATITUDE', 'LONGITUDE']] \
        if 'LATITUDE' in loc_data.columns and 'LONGITUDE' in loc_data.columns else pd.DataFrame()
    
    cord_data = cord_data[['LATITUDE', 'LONGITUDE']].drop_duplicates()
    
    loc_data = loc_data.merge(cord_data, on=['LATITUDE', 'LONGITUDE'], how='inner')
    
    return fac_data, loc_data, cord_data


In [6]:
def check_snf_conn():
    cur = conn.cursor()
    cur.execute("SELECT CURRENT_VERSION()")
    data = cur.fetchone()
    print("Snowflake Version:", data[0])
    cur.close()

In [7]:
def create_and_load_fac_table(data_col, fac_data, fac_tab):
    try:
        column_definitions = ', '.join([f'"{col}" STRING' for col in data_col])
        create_table_sql = f"""CREATE OR REPLACE TABLE {fac_tab} ({column_definitions});"""
        
        with conn.cursor() as cur:
            cur.execute(create_table_sql)
            print(f"Table {fac_tab} created successfully.")

        success, num_chunks, num_rows, num_cols = write_pandas(
            conn,
            fac_data,
            fac_tab  
        )
        print(f"Data loaded successfully: {num_rows} rows.")
    
    except Exception as e:
        print(f"Error: {e}")

In [8]:
def create_and_load_loc_table(fac_col, loc_data, loc_tab):
    try:
        column_definitions = ', '.join([f'"{col}" STRING' for col in fac_col])
        create_table_sql = f"""CREATE OR REPLACE TABLE {loc_tab} ({column_definitions});"""
        with conn.cursor() as cur:
            cur.execute(create_table_sql)
            print(f"Table {loc_tab} created successfully.")
        success, num_chunks, num_rows, num_cols = write_pandas(
            conn,
            loc_data,
            loc_tab  
        )
        print(f"Data loaded successfully: {num_rows} rows.")
    except Exception as e:
        print(f"Error: {e}")

In [9]:
def create_and_load_cord_table(cord_col, cord_data, cord_tab):
    try:
        column_definitions = ', '.join([f'"{col}" STRING' for col in cord_col])
        create_table_sql = f"""CREATE OR REPLACE TABLE {cord_tab} ({column_definitions});"""
        with conn.cursor() as cur:
            cur.execute(create_table_sql)
            print(f"Table {cord_tab} created successfully.")
        success, num_chunks, num_rows, num_cols = write_pandas(
            conn,
            cord_data,
            cord_tab 
        )
        print(f"Data loaded successfully: {num_rows} rows.")
    except Exception as e:
        print(f"Error: {e}")

In [10]:
def display_all_table():
    def get_table_row_count(table_name):
        query = f"SELECT COUNT(*) FROM {table_name} "
        result = pd.read_sql(query, conn)  
        return result.iloc[0, 0]  
        
    def display_table(table_name):
        query = f"SELECT * FROM {table_name} LIMIT 10;"
        result = pd.read_sql(query, conn)  
        return result  
        
    try:
        try:
            fac_count = get_table_row_count(f'{fac_tab}')
            print(f"Row count for {fac_tab}: {fac_count}")
            print(display_table(f'{fac_tab}'))
        except Exception as e:
            print(f"Error retrieving row count for {fac_tab}: {e}")
        try:
            loc_count = get_table_row_count(f'{loc_tab}')
            print(f"Row count for {loc_tab}: {loc_count}")
            print(display_table(f'{loc_tab}'))
        except Exception as e:
            print(f"Error retrieving row count for {loc_tab}: {e}")
        try:
            cord_count = get_table_row_count(f'{cord_tab}')
            print(f"Row count for {cord_tab}: {cord_count}")
            print(display_table(f'{cord_tab}'))
        except Exception as e:
            print(f"Error retrieving row count for {cord_tab}.lower(): {e}") 
    except Exception as e:
        print(f"Error: {e}")
    finally:
        conn.close()

In [11]:
check_snf_conn()
num_rows = ups_facilities.shape[0]
print(f"Total rows count in CSV: {num_rows}")

fac_data, loc_data, cord_data = pre_process_locations(ups_facilities)

fac_col = fac_data.columns
create_and_load_fac_table(fac_col, fac_data, fac_tab)

loc_col = loc_data.columns
create_and_load_loc_table(loc_col, loc_data, loc_tab)

cord_col = cord_data.columns
create_and_load_cord_table(cord_col, cord_data, cord_tab)

Snowflake Version: 8.34.0
Total rows count in CSV: 49317
Table FAC_DATA_TB created successfully.
Data loaded successfully: 49317 rows.
Table FAC_LOC_TB created successfully.
Data loaded successfully: 6362 rows.
Table FAC_CORD_TB created successfully.
Data loaded successfully: 6362 rows.


In [12]:
display_all_table()
conn.close()

Row count for FAC_DATA_TB: 49317
  index              X             Y   FID FEATURE_ID                   NAME  \
0     0  -77.896729019  34.042372977  4001      21735           UPS Drop Box   
1     1  -78.400208023  33.957407991  4002      21739           UPS Drop Box   
2     2  -78.531433999  35.949646009  4003      14826           UPS Drop Box   
3     3  -78.383909994   33.97427499  4004      21740           UPS Drop Box   
4     4  -77.966663998     35.973269  4005      14827           UPS Drop Box   
5     5  -78.405726989  33.961218015  4006      21741           UPS Drop Box   
6     6  -78.541932988  35.953805998  4007      14828           UPS Drop Box   
7     7  -78.371873996  33.980735988  4008      21742           UPS Drop Box   
8     8  -78.543679994  35.978425021  4009      14829           UPS Drop Box   
9     9  -78.374769012  33.978475999  4010      21743  UPS Alliance Location   

                       ADDRESS ADDRESS2 ADDRESS3            CITY STATE    ZIP  \
0    